In [ ]:
import pandas as pd
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
import tensorflow as tf
import re
import requests
import glob
import json
from sklearn.metrics import roc_curve, auc, precision_recall_curve, average_precision_score
from sklearn.preprocessing import label_binarize
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split
from dataclasses import dataclass, field
from sparrow import Protein
from collections import defaultdict
from Bio.PDB import PDBParser
from Bio.PDB.DSSP import DSSP

## Input staging function Declarations

### Tripeptide Column Generation

In [ ]:
def getTripeptideColumn(df, ordered_list, disordered_list):
    '''
    Input: Dataframe containing the Columns: ID with PDB IDs, Residue Number containing position in the peptide,
           amino_acid containing the AA at that position and Disordered containing a binary value 

           List of Ordered statistically significant tripeptide motifs

           List of Disordered statistically significant tripeptide motifs

    Output: Input dataframe with a tripeptide significance column appended containing an array with binary values
            indicating whether all possible tripeptides containing that positions amino acid statistical significance

    Function: This function runs along every amino acid in a peptide and checks all tripeptides that contain it for 
              statistical significance against the lists provided 
    '''
    Significance_df = pd.DataFrame()
    # Iterate over rows in the data frame
    uniprot_ids = sorted(set(df.loc[:,'ID']))
    for uniprot_id in uniprot_ids:
        uniprot_df = df[df['ID'] == uniprot_id]
        residue_numbers = uniprot_df['residue_number'].values
        for i in uniprot_df.index:
            tripeptides = list()
            if uniprot_df.loc[i, 'Disordered'] == 1:
                tripeptide_list = disordered_list

            elif uniprot_df.loc[i, 'Disordered'] == 0:
                tripeptide_list = ordered_list

            else:
                tripeptide_list = ordered_list
    # First Amino Acid
            if i == uniprot_df.index[0]:
                    """
                    [A]-T-C-Y-K-P-Y-L
                         0  1 2 3 4 5 6 7
                    
                    Should only return ATC
                    
                    """
                    tripeptide = uniprot_df.loc[i:i+2,'amino_acid'].values
                    if len(tripeptide) != 3: 
                        sys.stderr.write(f"{tripeptide}\tERROR!!!! at position {i} in {uniprot_df.loc[i, 'ID']}\n")
                    else:
                        tripeptide = [''.join(tripeptide)]
                        if tripeptide in tripeptide_list:
                            uniprot_df.at[i, 'tripeptide_significance'] = [0,0,1]

                        else:
                            uniprot_df.at[i, 'tripeptide_significance'] = [0,0,0]

    # Second Amino Acid
            elif i == uniprot_df.index[1]:
                    """ 
                    A-[T]-C-Y-K-P-Y-L
                         0  1  2 3 4 5 6 7
                    
                    Should only return ATC and TCY
                    
                    """
                    
                    tripeptide = uniprot_df.loc[i-1:i+1,'amino_acid'].values
                    if len(tripeptide) != 3: 
                        sys.stderr.write(f"{tripeptide}\tERROR!!!! at position {i} in {uniprot_df.loc[i, 'ID']}\n")
                    else:
                        tripeptide = [''.join(tripeptide)]
                        if tripeptide in tripeptide_list:
                            uniprot_df.at[i, 'tripeptide_significance'] = [0,1]

                        else:
                            uniprot_df.at[i, 'tripeptide_significance'] = [0,0]
                    
                    tripeptide = uniprot_df.loc[i:i+2,'amino_acid'].values
                    
                    if len(tripeptide) != 3: 
                        sys.stderr.write(f"{tripeptide}\tERROR!!!! at position {i} in {uniprot_df.loc[i, 'ID']}\n")
                    else:
                        tripeptide = [''.join(tripeptide)]
                        if tripeptide in tripeptide_list:
                            uniprot_df.loc[i, 'tripeptide_significance'].append(1)

                        else:
                            uniprot_df.loc[i, 'tripeptide_significance'].append(0)

    # Last Amino Acid
            elif i == uniprot_df.index[-1]:
                    
                    """  A-T-C-Y-K-P-Y-[L]
                         0 1 2 3 4-3-2 -1
                         
                    Should only return PYL
                    
                    """
                    tripeptide = uniprot_df.loc[i-2:i,'amino_acid'].values
                    
                    if len(tripeptide) != 3: 
                        sys.stderr.write(f"{tripeptide}\tERROR!!!! at position {i} in {uniprot_df.loc[i, 'ID']}\n")
                    else:
                        tripeptide = [''.join(tripeptide)]
                        if tripeptide in tripeptide_list:
                            uniprot_df.at[i, 'tripeptide_significance'] = [1,0,0]

                        else:
                            uniprot_df.at[i, 'tripeptide_significance'] = [0,0,0]

    # Second Last Amino Acid
            elif i == uniprot_df.index[-2]:
                    
                    """  A-T-C-Y-K-P-[Y]-L
                         0 1 2 3 4-3 -2 -1
                         
                    Should only return PYL and KPY
                    
                    """
                    tripeptide = uniprot_df.loc[i-2:i,'amino_acid'].values
                    
                    if len(tripeptide) != 3: 
                        sys.stderr.write(f"{tripeptide}\tERROR!!!! at position {i} in {uniprot_df.loc[i, 'ID']}\n")
                    else:
                        tripeptide = [''.join(tripeptide)]
                        if tripeptide in tripeptide_list:
                            uniprot_df.loc[i, 'tripeptide_significance'] = [1]

                        else:
                            uniprot_df.loc[i, 'tripeptide_significance'] = [0]
                    
                    tripeptide = uniprot_df.loc[i-1:i+1,'amino_acid'].values
                    
                    if len(tripeptide) != 3: 
                        sys.stderr.write(f"{tripeptide}\tERROR!!!! at position {i} in {uniprot_df.loc[i, 'ID']}\n")
                    else:
                        tripeptide = [''.join(tripeptide)]
                        if tripeptide in tripeptide_list:
                            uniprot_df.loc[i, 'tripeptide_significance'].append(1)
                            uniprot_df.loc[i, 'tripeptide_significance'].append(0)
                            

                        else:
                            uniprot_df.loc[i, 'tripeptide_significance'].append(0)
                            uniprot_df.loc[i, 'tripeptide_significance'].append(0)
    # All Other AAs
            else: 
                    
                    tripeptide = uniprot_df.loc[i-2:i,'amino_acid'].values
                    
                    if len(tripeptide) != 3: 
                        sys.stderr.write(f"{tripeptide}\tERROR!!!! at position {i} in {uniprot_df.loc[i, 'ID']}\n")
                    else:
                        tripeptide = [''.join(tripeptide)]
                        if tripeptide in tripeptide_list:
                            uniprot_df.loc[i, 'tripeptide_significance'] = [1]

                        else:
                            uniprot_df.loc[i, 'tripeptide_significance'] = [0]
                        
                    tripeptide = uniprot_df.loc[i-1:i+1,'amino_acid'].values
                    if len(tripeptide) != 3: 
                        sys.stderr.write(f"{tripeptide}\tERROR!!!! at position {i} in {uniprot_df.loc[i, 'ID']}\n")
                    else:
                        tripeptide = [''.join(tripeptide)]
                        if tripeptide in tripeptide_list:
                            uniprot_df.loc[i, 'tripeptide_significance'].append(1)

                        else:
                            uniprot_df.loc[i, 'tripeptide_significance'].append(0)
                        
                    tripeptide = uniprot_df.loc[i:i+2,'amino_acid'].values
                    if len(tripeptide) != 3: 
                        sys.stderr.write(f"{tripeptide}\tERROR!!!! at position {i} in {uniprot_df.loc[i, 'ID']}\n")
                    else:
                        tripeptide = [''.join(tripeptide)]
                        if tripeptide in tripeptide_list:
                            uniprot_df.loc[i, 'tripeptide_significance'].append(1)

                        else:
                            uniprot_df.loc[i, 'tripeptide_significance'].append(0)
        
        Significance_df = pd.concat([Significance_df ,uniprot_df], ignore_index=True)

    return(Significance_df)
                        

### Amino Acid Vector Generation

In [ ]:
def getAA_column(df):
    '''
    Input: Dataframe containing the Columns: ID with PDB IDs, Residue Number containing position in the peptide,
           amino_acid containing the AA at that position 

    Output: Input Dataframe with column containing a vector representing that rows amino acid in one hot encoding

    Function: Take all rows amino acid a generate a vector representing that amino acid in one hot encoding 
    '''

    vectorised_df = pd.DataFrame()
    # Iterate over rows in the data frame
    uniprot_ids = sorted(set(df.loc[:,'ID']))
    for uniprot_id in uniprot_ids:
        uniprot_df = df[df['ID'] == uniprot_id]
        residue_numbers = uniprot_df['residue_number'].values
        for i in uniprot_df.index:
    # Positively Charged AAs
    # Arginine - Pos 1
            if uniprot_df.loc[i, 'amino_acid'] == 'R':
                i_list = [1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
                i_vector = np.array(i_list)
                uniprot_df.at[i, 'amino_acid_vector'] = i_vector

    # Histidine - Pos 2
            elif uniprot_df.loc[i, 'amino_acid'] == 'H':
                i_list = [0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
                i_vector = np.array(i_list)
                uniprot_df.at[i, 'amino_acid_vector'] = i_vector
                
    # Lysine - Pos 3
            elif uniprot_df.loc[i, 'amino_acid'] == 'K':
                i_list = [0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
                i_vector = np.array(i_list)
                uniprot_df.at[i, 'amino_acid_vector'] = i_vector

    # Negatively Charged AAs
    # Aspartic Acid - Pos 4
            elif uniprot_df.loc[i, 'amino_acid'] == 'D':
                i_list = [0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
                i_vector = np.array(i_list)
                uniprot_df.at[i, 'amino_acid_vector'] = i_vector

    # Glutamic Acid - Pos 5
            elif uniprot_df.loc[i, 'amino_acid'] == 'E':
                i_list = [0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
                i_vector = np.array(i_list)
                uniprot_df.at[i, 'amino_acid_vector'] = i_vector

    # Polar Uncharged AAs
    # Serine - Pos 6
            elif uniprot_df.loc[i, 'amino_acid'] == 'S':
                i_list = [0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
                i_vector = np.array(i_list)
                uniprot_df.at[i, 'amino_acid_vector'] = i_vector

    # Threonine - Pos 7 
            elif uniprot_df.loc[i, 'amino_acid'] == 'T':
                i_list = [0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0]
                i_vector = np.array(i_list)
                uniprot_df.at[i, 'amino_acid_vector'] = i_vector

    # Asparagine  - Pos 8 
            elif uniprot_df.loc[i, 'amino_acid'] == 'N':
                i_list = [0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0]
                i_vector = np.array(i_list)
                uniprot_df.at[i, 'amino_acid_vector'] = i_vector

    # Glutamine - Pos 9 
            elif uniprot_df.loc[i, 'amino_acid'] == 'Q':
                i_list = [0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0]
                i_vector = np.array(i_list)
                uniprot_df.at[i, 'amino_acid_vector'] = i_vector

    # Special cases - come back to this
    # Glycine - Pos 10
            elif uniprot_df.loc[i, 'amino_acid'] == 'G':
                i_list = [0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0]
                i_vector = np.array(i_list)
                uniprot_df.at[i, 'amino_acid_vector'] = i_vector

    # Proline - Pos 11
            elif uniprot_df.loc[i, 'amino_acid'] == 'P':
                i_list = [0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0]
                i_vector = np.array(i_list)
                uniprot_df.at[i, 'amino_acid_vector'] = i_vector

    # Cysteine - Pos 12
            elif uniprot_df.loc[i, 'amino_acid'] == 'C':
                i_list = [0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0]
                i_vector = np.array(i_list)
                uniprot_df.at[i, 'amino_acid_vector'] = i_vector

    # Hydrophobic AAs
    # Alanine - Pos 13
            elif uniprot_df.loc[i, 'amino_acid'] == 'A':
                i_list = [0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0]
                i_vector = np.array(i_list)
                uniprot_df.at[i, 'amino_acid_vector'] = i_vector

    # Valine - Pos 14
            elif uniprot_df.loc[i, 'amino_acid'] == 'V':
                i_list = [0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0]
                i_vector = np.array(i_list)
                uniprot_df.at[i, 'amino_acid_vector'] = i_vector

    # Isoleucine - Pos 15
            elif uniprot_df.loc[i, 'amino_acid'] == 'I':
                i_list = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0]
                i_vector = np.array(i_list)
                uniprot_df.at[i, 'amino_acid_vector'] = i_vector

    # Leucine - Pos 16
            elif uniprot_df.loc[i, 'amino_acid'] == 'L':
                i_list = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0]
                i_vector = np.array(i_list)
                uniprot_df.at[i, 'amino_acid_vector'] = i_vector

    # Methionine - Pos 17
            elif uniprot_df.loc[i, 'amino_acid'] == 'M':
                i_list = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0]
                i_vector = np.array(i_list)
                uniprot_df.at[i, 'amino_acid_vector'] = i_vector

    # Phenylalanine - Pos 18
            elif uniprot_df.loc[i, 'amino_acid'] == 'F':
                i_list = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0]
                i_vector = np.array(i_list)
                uniprot_df.at[i, 'amino_acid_vector'] = i_vector
        
    # Tyrosine - Pos 19
            elif uniprot_df.loc[i, 'amino_acid'] == 'Y':
                i_list = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0]
                i_vector = np.array(i_list)
                uniprot_df.at[i, 'amino_acid_vector'] = i_vector

    # Tryptophan - Pos 20
            elif uniprot_df.loc[i, 'amino_acid'] == 'W':
                i_list = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1]
                i_vector = np.array(i_list)
                uniprot_df.at[i, 'amino_acid_vector'] = i_vector

    # Empty Position - Pos Null
            elif uniprot_df.loc[i, 'amino_acid'] == '-':
                i_list = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
                i_vector = np.array(i_list)
                uniprot_df.at[i, 'amino_acid_vector'] = i_vector


            else:
                print("Mahoraga HELP ME")

        vectorised_df = pd.concat([vectorised_df ,uniprot_df], ignore_index=True)
    return vectorised_df

### Generate Ground truth binding binary value

In [ ]:
def getGT_column(df):
    '''
    Input: Dataframe containing the Columns: ID with PDB IDs, Residue Number containing position in the peptide
           and a column containing a givens rows amino acids distance to RNA

    Output: Input dataframe with column appended containing binary binding value

    Function: Take the distance to RNA column and generate a binding binary vector based on whether its below 
              the threshold of 4.2
    '''
    GT_df = pd.DataFrame()
    # Iterate over rows in the data frame
    uniprot_ids = sorted(set(df.loc[:,'ID']))
    for uniprot_id in uniprot_ids:
        uniprot_df = df[df['ID'] == uniprot_id]
        residue_numbers = uniprot_df['residue_number'].values
        for i in uniprot_df.index:
            if uniprot_df.loc[i, 'Distance_to_RNA'] > 4.2:
                uniprot_df.at[i, 'binding_ground_truth'] = [0]

            elif uniprot_df.loc[i, 'Distance_to_RNA'] <= 4.2:
                uniprot_df.at[i, 'binding_ground_truth'] = [1]

            else:
                sys.stderr.write(f"Non-Integer Input within Distance_to_RNA column")

        GT_df = pd.concat([GT_df, uniprot_df], ignore_index=True)

    return GT_df
            

### Disorder Prediction using Albatross and Sparrow Package

In [ ]:
def getDisorder_column(df, do_GT):
    '''
    Input: Dataframe containing columns: ID with Uniprot IDs, Residue number containing sequence position
           , Amino acid containing an Amino acid letter and Disorder which should be NA 

    Output: Binary disorder prediction replaces a given positions NA disorder value

    Function: this function takes the sequence of a given peptide and uses Albatross to 
              predict the likelihood of a given amino acid in the sequence being a component
              of a disordered domain and then tagging it as 0 or 1 based on a threshhold 
              value of 0.6

    '''
    Diso_df = pd.DataFrame()
    # Iterate over rows in the data frame
    uniprot_ids = sorted(set(df.loc[:,'ID']))
    for uniprot_id in uniprot_ids:
        uniprot_df = df[df['ID'] == uniprot_id]
        residue_numbers = uniprot_df['residue_number'].values

        uniprot_seq = uniprot_df['amino_acid'].str.cat()
        uniprot_prediction = Protein(uniprot_seq)
        uniprot_df = uniprot_df.reset_index()
        if do_GT == True:
            for i in uniprot_df.index:
            # currently using binary predict with the threshhold at 0.6
            # tighten this later but works for now - close enough to uniprot id features on selected AAs
                uniprot_df.at[i, 'Disordered'] =  uniprot_prediction.predictor.binary_disorder(disorder_threshold=0.6)[i]
                uniprot_df.at[i, 'disorder_propensity'] = uniprot_prediction.predictor.disorder()[i]

        if do_GT == False:
            for i in uniprot_df.index:
                uniprot_df.at[i, 'disorder_propensity'] = uniprot_prediction.predictor.disorder()[i]
            

        Diso_df = pd.concat([Diso_df, uniprot_df], ignore_index=True)

    return Diso_df
            

### Pull Alphafold models for specified IDs

In [ ]:
def downloadAlphaFoldModel(uniprot_id,file_type='pdb',out_dir='PDB_files'):
    """Downloads the alpha for the specified model from PDBe."""
    url = f"https://alphafold.ebi.ac.uk/api/prediction/{uniprot_id}"
    headers={"Content-Type":"text"}
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        results_dict = json.loads(response.content)[0]
        entryid = results_dict['entryId']
        out_file_name = f"{entryid}.{file_type}"
        out_file_folder = f"{out_dir}/{uniprot_id}/"
        out_file_path = f"{out_dir}/{uniprot_id}/{out_file_name}"
        if file_type == 'pdb':
            download_link = results_dict['pdbUrl']
            model = requests.get(download_link, headers=headers)
            if model.status_code == 200:
                if not os.path.exists(out_dir):
                    os.makedirs(out_dir)

                if not os.path.exists(out_file_folder):
                    os.makedirs(out_file_folder)
                with open(out_file_path,"wb") as pdb:
                    pdb.write(model.content)
                return True
            else:
                sys.stderr.write(f"Failed to download alphafold model for {uniprot_id}: {response.status_code}\n")
                return False
    else:
        sys.stderr.write(f"Failed to download alphafold model for {uniprot_id}: {response.status_code}\n")        
        return False

### Use DSSP to get accessibility and secondary structure from downloaded Alphafold models

In [ ]:
def getDSSPinformation(df):

    dssp_table = pd.DataFrame(columns=['ID','pdb_id','residue_number','amino_acid','secondary_structure','accessibility','avg_b_factor'])

    merged_df = pd.DataFrame(columns=df.columns)
    
    parser = PDBParser()
    
    uniprot_ids = sorted(set(df.loc[:,'ID']))
    for uniprot_id in uniprot_ids:
        uniprot_df = df[df['ID'] == uniprot_id]

        results_df = pd.DataFrame(columns=['ID','pdb_id','residue_number', 'amino_acid', 'secondary_structure', 'accessibility', 'avg_b_factor'])

        model_bool = downloadAlphaFoldModel(uniprot_id,file_type='pdb')
        
        if model_bool == True:
            structure_path_pattern = f'PDB_files/{uniprot_id}/*'
            pdb_file_path = glob.glob(structure_path_pattern)
            
            if len(pdb_file_path) > 0: 
                id_structure = parser.get_structure(f'{uniprot_id}', pdb_file_path[0])
                id_model = id_structure[0]
            
            try:
                id_dssp = DSSP(id_model, pdb_file_path[0])
            except:
                sys.stderr.write(f"Could not create DSSP data for {uniprot_id}\n")
            else:
                for key in id_dssp.keys():
                    index, amino_acid, secondary_structure, accessibility = id_dssp[key][0], id_dssp[key][1], id_dssp[key][2], id_dssp[key][3]
                    chain_id, res_id = key
                    residue = id_model[chain_id][res_id]
                
                    b_factors = [atom.get_bfactor() for atom in residue if atom.element != 'H']  # Exclude hydrogen atoms
                    avg_b_factor = sum(b_factors) / len(b_factors) if b_factors else None
                    
                    new_row = pd.DataFrame({
                        'ID': [uniprot_id],
                        'pdb_id': [uniprot_id],
                        'residue_number': [index], 
                        'amino_acid': [amino_acid], 
                        'secondary_structure': [secondary_structure],
                        'accessibility': [accessibility],
                        'avg_b_factor': [avg_b_factor]
                    })
    
                    
                    results_df = pd.concat([results_df, new_row], ignore_index=True)
            dssp_table = pd.concat([dssp_table,results_df])

        elif model_bool == False:
            sys.stdout.write(f'{uniprot_id} model not found: passing to next ID')
            pass


        dssp_results = dssp_table.loc[dssp_table['ID'] == uniprot_id]
        if not dssp_results.empty:
        # Ensure selection uses 'data_table' to match 'uniprot_id'
            selection = df.loc[df['ID'] == uniprot_id].copy()  # Correct conditional source
            if not selection.empty:
                for i in selection.index:
                    residue_number = selection.loc[i, 'residue_number']
                    amino_acid = selection.loc[i, 'amino_acid']

                # Extract accessibility values safely
                    accessibility_values = dssp_results.loc[
                        (dssp_results['residue_number'] == residue_number) &
                        (dssp_results['amino_acid'] == amino_acid),
                        'accessibility'
                    ].values

                # Check if values are found before assignment
                    if accessibility_values.size > 0:
                        accessibility = accessibility_values[0]
                        if accessibility:
                            selection.loc[i, 'accessibility'] = accessibility
                    else:
                        sys.stderr.write(f"No data found for uniprot {uniprot_id}, residue {residue_number} AA {amino_acid}\n")
                    
            # Concatenate the modified `selection` DataFrame to `merged_expression_data`
            merged_df = pd.concat([merged_df, selection], ignore_index=True)
    return merged_df  
            

In [ ]:
def getSSvector(df):
    df['ss_vector'] = pd.NA
    secondary_structure_df = pd.DataFrame()
    uniprot_ids = sorted(set(df.loc[:,'ID']))
    for uniprot_id in uniprot_ids:
        uniprot_df = df[df['ID'] == uniprot_id]
        for i in uniprot_df.index:        
    
                    # Alpha Helix - H
            if uniprot_df.loc[i, 'secondary_structure'] == 'H':
                i_list = [1,0,0,0,0,0,0,0]
                i_vector = np.array(i_list)
                uniprot_df.at[i, 'ss_vector'] = i_vector 
    
                # Isolated Beta-Bridge Residue - B
            elif uniprot_df.loc[i, 'secondary_structure'] == 'B':
                i_list = [0,1,0,0,0,0,0,0]
                i_vector = np.array(i_list)
                uniprot_df.at[i, 'ss_vector'] = i_vector
    
                    # Strand
            elif uniprot_df.loc[i, 'secondary_structure'] == 'E':
                i_list = [0,0,1,0,0,0,0,0]
                i_vector = np.array(i_list)
                uniprot_df.at[i, 'ss_vector'] = i_vector
    
                    # 3-10 Helix - G
            elif uniprot_df.loc[i, 'secondary_structure'] == 'G':
                i_list = [0,0,0,1,0,0,0,0]
                i_vector = np.array(i_list)
                uniprot_df.at[i, 'ss_vector'] = i_vector
    
                    # Pi Helix - I
            elif uniprot_df.loc[i, 'secondary_structure'] == 'I':
                i_list = [0,0,0,0,1,0,0,0]
                i_vector = np.array(i_list)
                uniprot_df.at[i, 'ss_vector'] = i_vector
    
                    # Turn - T
            elif uniprot_df.loc[i, 'secondary_structure'] == 'T':
                i_list = [0,0,0,0,0,1,0,0]
                i_vector = np.array(i_list)
                uniprot_df.at[i, 'ss_vector'] = i_vector
    
    
                    # Bend - S
            elif uniprot_df.loc[i, 'secondary_structure'] == 'S':
                i_list = [0,0,0,0,0,0,1,0]
                i_vector = np.array(i_list)
                uniprot_df.at[i, 'ss_vector'] = i_vector
    
                    # ??? (Ask Sander) - P
            elif uniprot_df.loc[i, 'secondary_structure'] == 'P':
                i_list = [0,0,0,0,0,0,0,1]
                i_vector = np.array(i_list)
                uniprot_df.at[i, 'ss_vector'] = i_vector
    
                    # - - None
            elif uniprot_df.loc[i, 'secondary_structure'] == '-':
                i_list = [0,0,0,0,0,0,0,0]
                i_vector = np.array(i_list)
                uniprot_df.at[i, 'ss_vector'] = i_vector
    
            else:
                print(f'{uniprot_id} error')

        secondary_structure_df = pd.concat([secondary_structure_df ,uniprot_df], ignore_index=True)

    return secondary_structure_df


### Dataframe expansion for Model Input

In [ ]:
def pretrain_expand(df):
    '''

    '''
    # Assuming 'amino_acid_vector' needs to be expanded
    amino_acid_df = pd.DataFrame(df['amino_acid_vector'].tolist(), index=df.index)
    amino_acid_df.columns = [f'amino_acid_{i}' for i in range(1,len(amino_acid_df.columns)+1)]  # Optional: Naming columns

    tripeptide_df = pd.DataFrame(df['tripeptide_significance'].tolist(), index=df.index)
    tripeptide_df.columns = [f'tri_peptide_aa_{i}' for i in range(1,len(tripeptide_df.columns)+1)]  # Optional: Naming columns

    # Now join this back to the original dataframe if necessary
    df_expanded = df.join(amino_acid_df)
    df_expanded = df_expanded.join(tripeptide_df)
    df_expanded.drop(columns=['amino_acid_vector','tripeptide_significance'], axis=1, inplace=True)

    return df_expanded

### Generate Dataframe for Model training

In [ ]:
def training_data_preprocess(df, ordered_list, disordered_list, do_GT):


    if 'Disordered' in df.columns:
        pass

    else:
        df['Disordered'] = pd.NA
    if do_GT == True:
        req_columns = ["No", "ID", 'pdb_id', 'residue_number', 'amino_acid', 'Distance_to_RNA']

    elif do_GT == False:
        req_columns = ["No", "ID", 'residue_number', 'amino_acid', 'Disordered', 'RNA_binding']

    tripeptide_ready_data = df[req_columns]
    tripeptide_ready_data['tripeptide_significance'] = pd.NA
    tripeptide_ready_data['amino_acid_vector'] = pd.NA
    tripeptide_ready_data['accessibility'] = pd.NA
    tripeptide_ready_data['disorder_propensity'] = pd.NA
    if do_GT == True:
        tripeptide_ready_data['RNA_binding'] = pd.NA

    if do_GT == True:
        tripeptide_ready_data['Disordered'] = pd.NA
        tripeptide_ready_data = getDisorder_column(tripeptide_ready_data, True)

    if do_GT == False:
        tripeptide_ready_data = getDisorder_column(tripeptide_ready_data, False)
        tripeptide_ready_data['binding_ground_truth'] = tripeptide_ready_data['RNA_binding']
        tripeptide_ready_data.drop(columns=['RNA_binding'], axis=1, inplace=True)
        
    post_tripeptide_df = getTripeptideColumn(tripeptide_ready_data, ordered_list, disordered_list)

    aa_vectorised_df = getAA_column(post_tripeptide_df)

    if do_GT == True:
        gt_df = getGT_column(aa_vectorised_df)

    if do_GT == False:
        gt_df = aa_vectorised_df

    finalised_df = getDSSPinformation(gt_df)
    
    resorted_finalsed_df = finalised_df.sort_values(by='No', ascending = True)

    final_columns = ['ID','amino_acid_vector', 'tripeptide_significance', 'disorder_propensity', 'binding_ground_truth','accessibility']
    
    preExpand_df = resorted_finalsed_df[final_columns]

    model_input_final = pretrain_expand(preExpand_df)

    print(f'Total number of rows with NA values: {len(model_input_final)}')

    model_input_final.dropna(inplace=True)

    print(f'Total number of rows without NA values: {len(model_input_final)}')

    return model_input_final

### Input Vector Staging

In [ ]:
training_data = pd.read_csv('All_combined_results.txt', sep='\t')

In [ ]:
top20_tripep_ordered = pd.read_csv('Top20_tripeptides_ordered.csv')
top20_tripep_disordered = pd.read_csv('Top20_tripeptides_disordered.csv')

top20_tripep_ordered.rename(columns={'0': 'tripeptides'}, inplace= True)
top20_tripep_disordered.rename(columns={'0': 'tripeptides'}, inplace= True)

diso_significant_tripeptides = top20_tripep_disordered['tripeptides'].values
diso_significant_tripeptides = diso_significant_tripeptides.tolist()

ordered_significant_tripeptides = top20_tripep_ordered['tripeptides'].values
ordered_significant_tripeptides = ordered_significant_tripeptides.tolist()


In [ ]:
completed_input_df = training_data_preprocess(training_data, ordered_significant_tripeptides, diso_significant_tripeptides, True)

In [ ]:
completed_input_df

### Import of Disordered Test and Train Data

In [ ]:
def read_in_diso_data(training_data_path):
    rows = []
    with open(training_data_path, 'r') as file:
        data = file.read()
        lines = data.strip().split('\n')
        for i in range(0, len(lines), 6):
            PDB_ID = lines[i].strip('>')
            Protein_Seq = lines[i+1]
            Disordered = lines[i+2]
            RNA_binding = lines[i+5]

            for i in range(len(Protein_Seq)):
                rows.append({'ID': PDB_ID, 'residue_number': i+1, 'amino_acid': Protein_Seq[i], 'Disordered': int(Disordered[i]), 'RNA_binding': int(RNA_binding[i])})

    staged_data_df = pd.DataFrame(rows)

    staged_data_df = staged_data_df[staged_data_df['Disordered'] != 0]

    staged_data_df['No'] = range(len(staged_data_df))

    staged_data_df = staged_data_df.reset_index()

    return staged_data_df

In [ ]:
diso_training_data_df = read_in_diso_data('Diso_TrainingDataset.txt')

In [ ]:
diso_training_input_df = training_data_preprocess(diso_training_data_df, ordered_significant_tripeptides, diso_significant_tripeptides, False)

In [ ]:
diso_test_data_df = read_in_diso_data('TestDataset.txt')

In [ ]:
diso_test_input_df = training_data_preprocess(diso_test_data_df, ordered_significant_tripeptides, diso_significant_tripeptides, False)

In [ ]:
concatenated_input_df = pd.concat([completed_input_df, diso_test_input_df, diso_training_input_df], ignore_index=True)

In [ ]:
concatenated_input_df.to_csv('Concatenated_input_data.txt', sep="\t")

In [ ]:
concatenated_input_df = pd.read_csv('Concatenated_input_data.txt', sep='\t')

In [ ]:
completed_input_df.to_csv('DSSP_rbs_only.txt', sep='\t')

In [ ]:
DSSP_rbs_only_df = pd.read_csv('DSSP_rbs_only.txt', sep='\t')

In [ ]:
DSSP_rbs_only_df

### Reshaping the data to make it compatible with a CNN:

In [ ]:
# Assuming df_expanded is your final DataFrame after preprocessing
features = concatenated_input_df.drop(['binding_ground_truth', 'ID'], axis=1)  # ID is also not a feature
labels = concatenated_input_df['binding_ground_truth']

# Convert DataFrame to numpy arrays
X = features.values
y = labels.values

# Reshape X to fit the CNN input shape (batch_size, height, width, channels)
# For 1D sequence data, you could treat each feature as a channel:
X = X.reshape((X.shape[0], 1, X.shape[1], 1))  # Height = 1, Width = number of features, Channels = 1

### Train Test Split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

### Diso Train Time!!!

In [ ]:
diso_model = Sequential([
    Conv2D(32, (1, 3), activation='relu', input_shape=(1, X_train.shape[2], 1)),  # Adjust the input shape and kernel
    MaxPooling2D(pool_size=(1, 2)),
    Flatten(),  # Flatten converts the 3D output to 1D for input to the fully connected layers
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')  # Assuming binary classification
])

diso_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Now, let's check the model summary to understand its architecture
diso_model.summary()

In [ ]:
history = diso_model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), batch_size=32)

### Evalutation

In [ ]:
import matplotlib.pyplot as plt

# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])  # Ensure 'accuracy' is the right key; sometimes it could be 'acc'
plt.plot(history.history['val_accuracy'])  # Likewise, confirm if it should be 'val_accuracy' or 'val_acc'
plt.title('Model 3 accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.savefig('Model_03_Accuracy.png')
plt.show()

In [ ]:
# Plot training & validation loss values
plt.plot(history.history['loss'])

plt.plot(history.history['val_loss'])
plt.title('Model 3 loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.savefig('Model_03_Loss.png')
plt.show()


In [ ]:
# Assume your model's prediction probabilities for the positive class are stored in y_scores
y_scores = diso_model.predict(X_test)

# Compute ROC Curve
fpr, tpr, _ = roc_curve(y_test, y_scores)
roc_auc = auc(fpr, tpr)

# Plot ROC Curve
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Model 3 Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.savefig('Model_03_ROC.png')
plt.show()


In [ ]:
# Compute Precision-Recall and plot curve
precision, recall, thresholds = precision_recall_curve(y_test, y_scores)
average_precision = average_precision_score(y_test, y_scores)

# Plot Precision-Recall curve
plt.figure()
plt.plot(recall, precision, color='blue', lw=2, label='Precision-Recall curve (area = %0.2f)' % average_precision)
plt.fill_between(recall, precision, alpha=0.2, color='blue')
plt.xlim([0.0, 1.05])
plt.ylim([0.0, 1.05])
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Model 3 Precision-Reclass recall curves')
plt.legend(loc="lower left")
plt.savefig('Model_03_recall.png')
plt.show()


### RBS Only Model

In [ ]:
# Assuming df_expanded is your final DataFrame after preprocessing
rbs_features = DSSP_rbs_only_df.drop(['binding_ground_truth', 'ID', 'Unnamed: 0'], axis=1)  # ID is also not a feature
rbs_labels = DSSP_rbs_only_df['binding_ground_truth']

# Convert DataFrame to numpy arrays
rbs_X = rbs_features.values
rbs_y = rbs_labels.values

# Reshape X to fit the CNN input shape (batch_size, height, width, channels)
# For 1D sequence data, you could treat each feature as a channel:
rbs_X = rbs_X.reshape((rbs_X.shape[0], 1, rbs_X.shape[1], 1))  # Height = 1, Width = number of features, Channels = 1

In [ ]:
rbs_X_train, rbs_X_test, rbs_y_train, rbs_y_test = train_test_split(rbs_X, rbs_y, test_size=0.3, random_state=42)

In [ ]:
rbs_model = Sequential([
    Conv2D(32, (1, 3), activation='relu', input_shape=(1, rbs_X_train.shape[2], 1)),  # Adjust the input shape and kernel
    MaxPooling2D(pool_size=(1, 2)),
    Flatten(),  # Flatten converts the 3D output to 1D for input to the fully connected layers
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')  # Assuming binary classification
])

rbs_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Now, let's check the model summary to understand its architecture
rbs_model.summary()

In [ ]:
rbs_history = rbs_model.fit(rbs_X_train, rbs_y_train, epochs=10, validation_data=(rbs_X_test, rbs_y_test), batch_size=32)

In [ ]:
# Assume your model's prediction probabilities for the positive class are stored in y_scores
rbs_y_scores = rbs_model.predict(rbs_X_test)

# Compute ROC Curve
fpr, tpr, _ = roc_curve(rbs_y_test, rbs_y_scores)
roc_auc = auc(fpr, tpr)

# Plot ROC Curve
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Model 3 Receiver Operating Characteristic RBS only')
plt.legend(loc="lower right")
plt.savefig('Model_03_ROC_rbs.png')
plt.show()


In [ ]:
# Compute Precision-Recall and plot curve
precision, recall, thresholds = precision_recall_curve(rbs_y_test, rbs_y_scores)
average_precision = average_precision_score(rbs_y_test, rbs_y_scores)

# Plot Precision-Recall curve
plt.figure()
plt.plot(recall, precision, color='blue', lw=2, label='Precision-Recall curve (area = %0.2f)' % average_precision)
plt.fill_between(recall, precision, alpha=0.2, color='blue')
plt.xlim([0.0, 1.05])
plt.ylim([0.0, 1.05])
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Model 3 Precision-Reclass recall curves RBS only')
plt.legend(loc="lower left")
plt.savefig('Model_03_recall_rbs.png')
plt.show()
